In [ ]:
import pandas as pd
df = pd.read_csv("cleaned_mental_health_data.csv")

In [ ]:
print(df["status"].value_counts())

status
Depression    8023
Suicidal      7700
Normal        6242
Anxiety        730
Name: count, dtype: int64


In [ ]:
df.isnull().sum()

,0
statement,0
status,0
cleaned_text,16
sentiment,1


In [ ]:
df.info

<bound method DataFrame.info of                                                statement      status  \
0                                             oh my gosh     Anxiety   
1      trouble sleeping, confused mind, restless hear...     Anxiety   
2      All wrong, back off dear, forward doubt. Stay ...     Anxiety   
3      I've shifted my focus to something else but I'...     Anxiety   
4      I'm restless and restless, it's been a month n...     Anxiety   
...                                                  ...         ...   
22690  TW: Selfharm, mention of suicide This is a thr...  Depression   
22691  I just SIT, mindless, wasting away Honestly, w...  Depression   
22692  I am a 34 year old woman who is a virgin. I ha...  Depression   
22693  No matter what I will feel just so down. Even ...  Depression   
22694  My life has taken a huge turn for the better. ...  Depression   

                                            cleaned_text  sentiment  
0                                                oh gosh     0.0000  
1      trouble sleeping confused mind restless heart ...    -0.7269  
2      wrong back dear forward doubt stay restless re...    -0.7351  
3      've shifted focus something else 'm still worried    -0.2960  
4                 'm restless restless 's month boy mean    -0.4939  
...                                                  ...        ...  
22690  tw selfharm mention suicide throwaway account ...    -0.8589  
22691  sit mindless wasting away honestly wtf wrong f...    -0.2764  
22692  34 year old woman virgin kissed guy nine year ...     0.0204  
22693  matter feel even everything good day keep phon...    -0.0478  
22694  life taken huge turn better though high school...        NaN  

[22695 rows x 4 columns]>

In [ ]:
df.tail()

,statement,status,cleaned_text,sentiment
22690,"TW: Selfharm, mention of suicide This is a thr...",Depression,tw selfharm mention suicide throwaway account ...,-0.8589
22691,"I just SIT, mindless, wasting away Honestly, w...",Depression,sit mindless wasting away honestly wtf wrong f...,-0.2764
22692,I am a 34 year old woman who is a virgin. I ha...,Depression,34 year old woman virgin kissed guy nine year ...,0.0204
22693,No matter what I will feel just so down. Even ...,Depression,matter feel even everything good day keep phon...,-0.0478
22694,My life has taken a huge turn for the better. ...,Depression,life taken huge turn better though high school...,NaN


In [ ]:
df = df.dropna(subset=["cleaned_text","sentiment"]).reset_index(drop=True)

In [ ]:
df.isnull().sum()

,0
statement,0
status,0
cleaned_text,0
sentiment,0


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    text = re.sub(r"[^\w\s']", "", text)
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return " ".join(tokens)

In [ ]:
label_mapping = {label: i for i, label in enumerate(df["status"].unique())}
df["label"] = df["status"].map(label_mapping)

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np


In [ ]:
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df["statement"].tolist(), df["label"].tolist(), test_size=0.3, random_state=42
)

In [ ]:
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42
)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), np.array(train_labels))).batch(8)
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), np.array(val_labels))).batch(8)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), np.array(test_labels))).batch(8)


In [ ]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_mapping))

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [ ]:
model.fit(train_dataset, validation_data=val_dataset, epochs=5)

Epoch 1/5
1985/1985 [==============================] - 571s 251ms/step - loss: 0.5153 - accuracy: 0.7586 - val_loss: 0.4595 - val_accuracy: 0.7878
Epoch 2/5
1985/1985 [==============================] - 478s 241ms/step - loss: 0.3752 - accuracy: 0.8305 - val_loss: 0.4596 - val_accuracy: 0.7934
Epoch 3/5
1985/1985 [==============================] - 478s 241ms/step - loss: 0.2751 - accuracy: 0.8891 - val_loss: 0.5397 - val_accuracy: 0.7860
Epoch 4/5
1985/1985 [==============================] - 478s 241ms/step - loss: 0.1785 - accuracy: 0.9316 - val_loss: 0.6922 - val_accuracy: 0.7925
Epoch 5/5
1985/1985 [==============================] - 477s 241ms/step - loss: 0.1194 - accuracy: 0.9574 - val_loss: 0.7722 - val_accuracy: 0.7904


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


model.save_pretrained("/content/drive/MyDrive/mental_health_bert")
tokenizer.save_pretrained("/content/drive/MyDrive/mental_health_bert")


('mental_health_bert/tokenizer_config.json',
 'mental_health_bert/special_tokens_map.json',
 'mental_health_bert/vocab.txt',
 'mental_health_bert/added_tokens.json')

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

426/426 [==============================] - 35s 82ms/step - loss: 0.7540 - accuracy: 0.7942
Test Accuracy: 79.42%


In [ ]:
import pickle

with open("dataset4.pkl", "rb") as f:
    df = pickle.load(f, encoding="latin1")


<ipython-input-17-f5df36c321c4>:4: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  df = pickle.load(f, encoding="latin1")


In [ ]:
df.shape

(58110, 4)

In [ ]:
df.head()

,statement,status,cleaned_text,sentiment
0,oh my gosh,Anxiety,oh gosh,0.0000
1,"trouble sleeping, confused mind, restless hear...",Anxiety,trouble sleeping confused mind restless heart ...,-0.7269
2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety,wrong back dear forward doubt stay restless re...,-0.7351
3,I've shifted my focus to something else but I'...,Anxiety,'ve shifted focus something else 'm still worried,-0.2960
4,"I'm restless and restless, it's been a month n...",Anxiety,'m restless restless 's month boy mean,-0.4939


In [ ]:
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments


In [ ]:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['status'])

In [ ]:
from sklearn.model_selection import train_test_split

train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df['cleaned_text'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

In [ ]:
train_texts, val_texts, test_texts = train_texts.tolist(), val_texts.tolist(), test_texts.tolist()
train_labels, val_labels, test_labels = train_labels.tolist(), val_labels.tolist(), test_labels.tolist()

In [ ]:
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = TFRobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(df['status'].value_counts()))


def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=128)

# Apply tokenization
train_encodings = tokenize_function(train_texts)
val_encodings = tokenize_function(val_texts)
test_encodings=tokenize_function(test_texts)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [ ]:
import tensorflow as tf
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), np.array(train_labels))).batch(8)
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), np.array(val_labels))).batch(8)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), np.array(test_labels))).batch(8)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [ ]:
model.fit(train_dataset, validation_data=val_dataset, epochs=5)

Epoch 1/5
5811/5811 [==============================] - 1352s 233ms/step - loss: 0.6985 - accuracy: 0.7335 - val_loss: 0.5645 - val_accuracy: 0.7785
Epoch 2/5
5811/5811 [==============================] - 1334s 230ms/step - loss: 0.5165 - accuracy: 0.8007 - val_loss: 0.5434 - val_accuracy: 0.7952
Epoch 3/5
5811/5811 [==============================] - 1333s 229ms/step - loss: 0.4131 - accuracy: 0.8405 - val_loss: 0.5569 - val_accuracy: 0.7966
Epoch 4/5
5811/5811 [==============================] - 1334s 230ms/step - loss: 0.3374 - accuracy: 0.8689 - val_loss: 0.5613 - val_accuracy: 0.8090
Epoch 5/5
5811/5811 [==============================] - 1333s 229ms/step - loss: 0.2836 - accuracy: 0.8911 - val_loss: 0.6237 - val_accuracy: 0.8019
